### Import

import library and data sets

In [1]:
import pandas as pd
consultant_profiles = pd.read_csv('consultant_profiles.csv')
historical_data = pd.read_csv('historical_data_df.csv')

Import the dictionnary

In [2]:
import sys
sys.path.append('/content/drive/My Drive/DRIVE DOUAA-YOUSRA/S3/recommendation system/')

from keyword_to_skills import keyword_to_skills

In [3]:
print(keyword_to_skills)

{'machine learning': 'Machine Learning', 'ml': 'Machine Learning', 'predictive modeling': 'Data Analysis', 'training models': 'Machine Learning', 'algorithm development': 'Machine Learning', 'ai models': 'Machine Learning', 'ai learning': 'Machine Learning', 'making predictions': 'Machine Learning', 'data-driven insights': 'Machine Learning', 'teaching computers': 'Machine Learning', 'learning algorithms': 'Machine Learning', 'smart systems': 'Machine Learning', 'self-learning software': 'Machine Learning', 'automated decision making': 'Machine Learning', 'ai tools': 'Machine Learning', 'pattern recognition': 'Machine Learning', 'data patterns': 'Machine Learning', 'automated predictions': 'Machine Learning', 'ai solutions': 'Machine Learning', 'smart predictions': 'Machine Learning', 'intelligent systems': 'Machine Learning', 'automating tasks': 'Machine Learning', 'smart algorithms': 'Machine Learning', 'training ai': 'Machine Learning', 'learning from data': 'Machine Learning', 'dat

### Data set manipulation

show consultant profiles data set

In [4]:
consultant_profiles.head(10)

,Name,Skills,Languages,Industry_Expertise,Years_of_Experience,Phone_Number
0,Consultant_1,"Software Development, Blockchain","English, Arabic, German",Education,8,213626497067
1,Consultant_2,"Blockchain, Data Engineering, Project Manageme...","German, Spanish",Finance,8,213516354563
2,Consultant_3,"Computer Vision, Machine Learning, Data Engine...",English,Manufacturing,6,213594877652
3,Consultant_4,"Project Management, Data Analysis, Deep Learning",German,"Finance, Education",10,213545258370
4,Consultant_5,"Project Management, NLP, Business Analysis, Bl...","German, Arabic","Technology, Manufacturing",7,213597173249
5,Consultant_6,"Big Data, Machine Learning, Data Analysis","Arabic, German, French",Healthcare,9,213550635764
6,Consultant_7,"Computer Vision, Deep Learning, Project Manage...","English, Spanish","Manufacturing, Retail",10,213699756499
7,Consultant_8,"Deep Learning, Blockchain",German,Manufacturing,10,213614429673
8,Consultant_9,"Data Engineering, Data Analysis","Arabic, French","Manufacturing, Retail",4,213683498546
9,Consultant_10,"Deep Learning, Business Analysis, Cybersecurit...","Spanish, Arabic",Healthcare,5,213743193441


calculate the average rating for each consultant the merge them to consultant_profiles data set



In [5]:
mean_ratings = historical_data.groupby('Consultant_Name')['Success_Rating'].mean()
mean_ratings = mean_ratings.reset_index()  # Convert to DataFrame to rename
mean_ratings.rename(columns={'Consultant_Name': 'Name'}, inplace=True)
max_rating = mean_ratings['Success_Rating'].max()
mean_ratings['Normalized_Rating'] = mean_ratings['Success_Rating'] / max_rating

# Step 5: Print the result
print(mean_ratings[['Name', 'Normalized_Rating']])


              Name  Normalized_Rating
0     Consultant_1           0.909645
1    Consultant_10           0.902741
2   Consultant_100           0.869036
3    Consultant_12           0.929949
4    Consultant_14           0.824365
..             ...                ...
71   Consultant_93           0.964467
72   Consultant_94           0.848731
73   Consultant_96           0.856853
74   Consultant_97           0.777665
75   Consultant_98           0.867005

[76 rows x 2 columns]


In [6]:
# Step 1: Merge the mean ratings with the consultant_profiles dataframe
consultant_profiles = consultant_profiles.merge(mean_ratings[['Name', 'Normalized_Rating']],
                                                on='Name',
                                                how='left')

# Step 2: Fill NaN values with 0.5 for consultants who don't have a rating
consultant_profiles['Normalized_Rating'].fillna(0.5, inplace=True)

# Step 3: Rename 'Normalized_Rating' column to 'Rate' as requested
consultant_profiles.rename(columns={'Normalized_Rating': 'Rate'}, inplace=True)

# Display the first 10 rows
consultant_profiles.head(10)


,Name,Skills,Languages,Industry_Expertise,Years_of_Experience,Phone_Number,Rate
0,Consultant_1,"Software Development, Blockchain","English, Arabic, German",Education,8,213626497067,0.909645
1,Consultant_2,"Blockchain, Data Engineering, Project Manageme...","German, Spanish",Finance,8,213516354563,0.500000
2,Consultant_3,"Computer Vision, Machine Learning, Data Engine...",English,Manufacturing,6,213594877652,0.818274
3,Consultant_4,"Project Management, Data Analysis, Deep Learning",German,"Finance, Education",10,213545258370,0.719797
4,Consultant_5,"Project Management, NLP, Business Analysis, Bl...","German, Arabic","Technology, Manufacturing",7,213597173249,0.891371
5,Consultant_6,"Big Data, Machine Learning, Data Analysis","Arabic, German, French",Healthcare,9,213550635764,0.948223
6,Consultant_7,"Computer Vision, Deep Learning, Project Manage...","English, Spanish","Manufacturing, Retail",10,213699756499,0.792555
7,Consultant_8,"Deep Learning, Blockchain",German,Manufacturing,10,213614429673,0.816244
8,Consultant_9,"Data Engineering, Data Analysis","Arabic, French","Manufacturing, Retail",4,213683498546,0.791878
9,Consultant_10,"Deep Learning, Business Analysis, Cybersecurit...","Spanish, Arabic",Healthcare,5,213743193441,0.902741


calculate the sentiment intensity of each project then get the average for each consultant and merge them to consultant_profiles data set

In [7]:
!pip install vaderSentiment

from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
# Step 1: Rename 'Consultant_Name' to 'Name' in historical_data
historical_data.rename(columns={'Consultant_Name': 'Name'}, inplace=True)
analyzer = SentimentIntensityAnalyzer()
# Step 2: Calculate sentiment scores for each comment in the historical data
historical_data['Sentiment_Score'] = historical_data['Feedback_Comments'].apply(
    lambda comment: analyzer.polarity_scores(comment)['compound'] if pd.notnull(comment) else 0
)

# Step 3: Group by Consultant_Name and calculate the mean sentiment score
sentiment_scores = historical_data.groupby('Name')['Sentiment_Score'].mean()
# Step 5: Merge the sentiment scores into consultant_profiles
consultant_profiles = consultant_profiles.merge(
    sentiment_scores, on='Name', how='left'
)
# Step 6: Replace NaN scores (for consultants with no comments) with 0
consultant_profiles['Sentiment_Score'] = consultant_profiles['Sentiment_Score'].fillna(0.5)

consultant_profiles.head(10)


,Name,Skills,Languages,Industry_Expertise,Years_of_Experience,Phone_Number,Rate,Sentiment_Score
0,Consultant_1,"Software Development, Blockchain","English, Arabic, German",Education,8,213626497067,0.909645,0.52670
1,Consultant_2,"Blockchain, Data Engineering, Project Manageme...","German, Spanish",Finance,8,213516354563,0.500000,0.50000
2,Consultant_3,"Computer Vision, Machine Learning, Data Engine...",English,Manufacturing,6,213594877652,0.818274,0.67050
3,Consultant_4,"Project Management, Data Analysis, Deep Learning",German,"Finance, Education",10,213545258370,0.719797,0.44870
4,Consultant_5,"Project Management, NLP, Business Analysis, Bl...","German, Arabic","Technology, Manufacturing",7,213597173249,0.891371,0.65970
5,Consultant_6,"Big Data, Machine Learning, Data Analysis","Arabic, German, French",Healthcare,9,213550635764,0.948223,0.38180
6,Consultant_7,"Computer Vision, Deep Learning, Project Manage...","English, Spanish","Manufacturing, Retail",10,213699756499,0.792555,0.62920
7,Consultant_8,"Deep Learning, Blockchain",German,Manufacturing,10,213614429673,0.816244,0.65970
8,Consultant_9,"Data Engineering, Data Analysis","Arabic, French","Manufacturing, Retail",4,213683498546,0.791878,0.85190
9,Consultant_10,"Deep Learning, Business Analysis, Cybersecurit...","Spanish, Arabic",Healthcare,5,213743193441,0.902741,0.62062


### Text preprocessing

import nltk library with its resources

In [8]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk import pos_tag

# Download NLTK resources
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')
nltk.download('punkt_tab')

# Initialize NLTK objects
lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words('english'))

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\hp\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\hp\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\hp\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\hp\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\hp\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


check years of experience with regulier expressions

In [9]:
import re
def extract_experience_criteria(client_input):
    # Check for years and comparison terms
    experience_criteria = re.search(
        r'(more than|or more|less than|or less)?\s*(\d+)\s*years?\s*(experience)?\s*(more than|or more|less than|or less)?',
        client_input.lower()
    )

    if experience_criteria:
        # Extract the number of years (group 2 contains the number)
        years = int(experience_criteria.group(2))

        # Initialize comparison to None
        comparison = None

        # Check the comparison term before the number
        if experience_criteria.group(1):
            comparison = experience_criteria.group(1)

        # Check the comparison term after the number
        if experience_criteria.group(4):
            comparison = experience_criteria.group(4)

        # Now, return the years and comparison
        if comparison == "more than":
            return years, "more than"  # > years
        elif comparison == "or more":
            return years, "or more"  # >= years
        elif comparison == "less than":
            return years, "less than"  # < years
        elif comparison == "or less":
            return years, "or less"  # <= years
        else:
            return years, "exact"  # == years
    return None, None


Text preprocessing with nltk

In [10]:
def preprocess_text_nltk(text):
    """
    Preprocesses text by normalizing, removing stopwords,
    performing lemmatization, handling negations, and mapping keywords using NLTK.

    Args:
        text (str): The input text to preprocess.

    Returns:
        str: The cleaned and normalized text.
    """

    # Sort keywords by length in descending order
    sorted_keywords = sorted(keyword_to_skills.keys(), key=len, reverse=True)

    # Replace phrases in the text with corresponding skills
    for phrase in sorted_keywords:
        if phrase in text:
            # If the value for the keyword is a list, replace the phrase with all options
            if isinstance(keyword_to_skills[phrase], list):
                options = " / ".join(keyword_to_skills[phrase])
                text = text.replace(phrase, options)
            else:
                text = text.replace(phrase, keyword_to_skills[phrase])

    # Tokenize the text
    tokens = word_tokenize(text.lower())

    # Initialize variables
    processed_tokens = []
    is_negated = False  # Flag to track negation

    # Process each token
    for token in tokens:
        # Check if token matches a keyword and replace if it does
        if token in keyword_to_skills:
            token = keyword_to_skills[token]

        # Check if token is a negation word
        '''if token in negation_words:
            is_negated = True
        elif token not in stop_words and token.isalnum():  # Remove stopwords and non-alphanumeric tokens
            # Lemmatize the token
            lemma = lemmatizer.lemmatize(token)
            # Add "not_" prefix if negation flag is True
            if is_negated:
                lemma = f"not_{lemma}"
                is_negated = False  # Reset the negation flag after handling
            processed_tokens.append(lemma)  # Append after final processing

    return " ".join(processed_tokens)'''
    return " ".join(tokens)



### Essentiel Function to detect *languages*

we need to ensure that consultants who speak only French are excluded. Consultants who speak French along with other languages should still be considered.

In [11]:
import re

# Language mapping
languages_mapping = {
    "english": "English",
    "french": "French",
    "spanish": "Spanish",
    "german": "German",
    "arabic": "Arabic"
}

# Regex pattern to detect negations followed by any combination of languages
negation_pattern = r"\b(?:not|no|never|doesn't|don't|shouldn't|cannot|mustn't|isn't|not able to)\s+(?:speak|talk|know)\s+((?:english|french|spanish|german|arabic)(?:\s*(?:,|\bor\b|and\b)\s*(?:english|french|spanish|german|arabic))*)\b"

def handle_complex_negations_with_regex(text, languages_mapping):
    """
    Handles complex negations using regex by replacing negated languages with 'not_language'.

    Args:
        text (str): Input text.
        languages_mapping (dict): Mapping of language keywords to language names.

    Returns:
        str: Updated text with negated languages replaced.
    """
    def replace_negation(match):
        # Extract the list of languages from the match group
        languages = re.split(r"\s*(?:,|\bor\b|and\b)\s*", match.group(1).lower())
        # Replace each language with its negated form
        negated_languages = [f"not_{languages_mapping[lang].lower()}" for lang in languages]
        return " ".join(negated_languages)

    # Replace negated languages in the text
    updated_text = re.sub(negation_pattern, replace_negation, text, flags=re.IGNORECASE)

    # Handle positive mentions of languages
    for lang_key, lang_value in languages_mapping.items():
        updated_text = re.sub(fr"\b{lang_key}\b", lang_value.lower(), updated_text, flags=re.IGNORECASE)

    return updated_text




### Matching function

In [12]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
def match_consultants(client_input, consultant_profiles):
    # Extract years and comparison type before processing text
    years, comparison = extract_experience_criteria(client_input)

    # Combine consultant data for comparison
    consultant_profiles["Combined_Data"] = (
        consultant_profiles["Skills"].str.lower() + " " +
        consultant_profiles["Industry_Expertise"].str.lower() + " " +
        consultant_profiles["Languages"].str.lower() + " " +
        consultant_profiles["Years_of_Experience"].astype(str)
    )

    #Preprocess client input
    client_description = handle_complex_negations_with_regex(client_input, languages_mapping)
    client_description = preprocess_text_nltk(client_description)

    # Filter consultants based on the experience criteria (before tokenization)
    filtered_consultants = consultant_profiles.copy()  # Create a copy to avoid modifying the original DataFrame
    if years is not None:
        if comparison == "more than":
            filtered_consultants = filtered_consultants[
                filtered_consultants["Years_of_Experience"] > years  # More than X
            ]
        elif comparison == "less than":
            filtered_consultants = filtered_consultants[
                filtered_consultants["Years_of_Experience"] < years  # Less than X
            ]
        elif comparison == "exact":
            filtered_consultants = filtered_consultants[
                filtered_consultants["Years_of_Experience"] == years  # Exact match to X
            ]
        elif comparison == "or more":
            filtered_consultants = filtered_consultants[
                filtered_consultants["Years_of_Experience"] >= years  # More or equal to X
            ]
        elif comparison == "or less":
            filtered_consultants = filtered_consultants[
                filtered_consultants["Years_of_Experience"] <= years  # Less or equal to X
            ]

    # Check if the filtered DataFrame is empty
    if filtered_consultants.empty:
        print("No consultants found matching the specified criteria.")
        return pd.DataFrame()  # Return an empty DataFrame

    # Initialize the TF-IDF Vectorizer
    vectorizer = TfidfVectorizer(stop_words='english')

    # Fit the vectorizer on both client input and consultant profiles' Combined Data
    vectorizer.fit([client_description] + filtered_consultants["Combined_Data"].tolist())  # Use filtered_consultants

    # Transform the client input and consultant profiles into TF-IDF vectors
    client_vector = vectorizer.transform([client_description])
    consultant_vectors = vectorizer.transform(filtered_consultants["Combined_Data"])  # Use filtered_consultants

    # Calculate cosine similarity between the client and each consultant's profile
    similarities1 = cosine_similarity(client_vector, consultant_vectors).flatten()

    # Use .loc to safely set the similarity score in the DataFrame
    filtered_consultants.loc[:, 'Similarity_Score'] = (similarities1 * 0.95) + (filtered_consultants['Rate'] * 0.025) + (filtered_consultants['Sentiment_Score'] * 0.025)  # Use filtered_consultants

    # Sort consultants by similarity score in descending order
    return filtered_consultants.sort_values(by="Similarity_Score", ascending=False)  # Use filtered_consultants

### the result

Check the work of text precessing in the client input

In [13]:
client_input = "Looking for a consultant  with more than 4 years  of expertise to creat a recommendation system in Healthcare but it is better if he doesn't speak arabic or german or french or arabic but speak spanish "
client_description = handle_complex_negations_with_regex(client_input, languages_mapping)
client_description = preprocess_text_nltk(client_description)
client_description

'looking for a consultant with more than 4 years of expertise to creat a data analysis / machine learning / software development in healthcare but it is better if he not_arabic not_german not_french not_arabic but speak spanish'

The final result

In [14]:
result = match_consultants(client_input, consultant_profiles)

pd.set_option('display.max_colwidth', None)
# Modify the "Similarity_Score" to be displayed as a percentage with one decimal place
result["Similarity_Score"] = (result["Similarity_Score"] * 100).round(1)

result[["Name", "Skills","Industry_Expertise","Years_of_Experience", "Languages","Similarity_Score"]].iloc[:10]

,Name,Skills,Industry_Expertise,Years_of_Experience,Languages,Similarity_Score
9,Consultant_10,"Deep Learning, Business Analysis, Cybersecurity, Data Engineering, Software Development",Healthcare,5,"Spanish, Arabic",24.2
12,Consultant_13,"Deep Learning, Cybersecurity, Software Development, NLP, Data Analysis",Healthcare,9,French,21.9
17,Consultant_18,"Data Engineering, Deep Learning, Computer Vision, Software Development",Healthcare,8,German,21.6
5,Consultant_6,"Big Data, Machine Learning, Data Analysis",Healthcare,9,"Arabic, German, French",21.3
25,Consultant_26,"Big Data, Project Management, Software Development, Cybersecurity, Data Analysis","Healthcare, Retail",8,"Arabic, German, Spanish",20.2
70,Consultant_71,"Machine Learning, Software Development, Computer Vision, Cloud Computing, Project Management","Finance, Healthcare",10,Spanish,20.0
48,Consultant_49,"NLP, Project Management, Software Development","Healthcare, Retail",6,"German, Spanish",19.5
85,Consultant_86,"Data Engineering, Software Development",Healthcare,6,"French, Arabic, English",19.3
74,Consultant_75,"Software Development, Machine Learning, Computer Vision, Big Data, Cybersecurity",Technology,8,"French, English",19.1
62,Consultant_63,"Big Data, Software Development","Manufacturing, Healthcare",7,"German, French, Arabic",18.8
